In [2]:
cd F:

F:\


In [43]:
from zoomtools import rdb, solr, transform
import pandas as pd
import itertools
import glob
from IPython import display as dsp
import os
import codecs
import numpy as np

In [45]:
# Dir on public drive where the clickback files live
data_dir =  "Z:\Katlyn.Donohue\Email Deliverability\Clickback\Weekly Clickback Files"

In [6]:
# Grabs all of the file names, which columns we want to read.
clickback_csv = [xml for xml in glob.iglob('zoominfo_permissionpass*.csv')]
clmns = ['Import_Date','Email_Health','Bounce_Code','Verification_Result','ZoomIndividualID','Email_Address']
clickback_csv[2]

'zoominfo_permissionpass_new_weekly_20160507_090000_Verified_20160530.csv'

**Take 1k sample from each file to compare against zipi_db to get Guessed/Evidence counts.**

In [60]:
_rows = None # # None for full read, smaller <N> for faster execution while debugging
records_per_file = 1000  # sample size to take from each file, higher for better confidence

CBsamples = pd.DataFrame()
counter = 0

for i in clickback_csv[:5]:
    dsp.clear_output
    print(counter)
    file = pd.read_csv(i, usecols=clmns, nrows=_rows)
    file = file.sample(records_per_file)
    CBsamples = CBsamples.append(file)
    counter +=1
    
CBsamples.head()


'\nfor i in clickback_csv[:5]:\n    dsp.clear_output\n    print(counter)\n    file = pd.read_csv(i, usecols=clmns, nrows=_rows)\n    file = file.sample(records_per_file)\n    CBsamples = CBsamples.append(file)\n    counter +=1\n    \nCBsamples.head()\n'

In [61]:
# Uncomment if saving for later. 
## CBsamples.to_csv('CBsamples.csv', index=False)

### Guessed or Evidence?

In [64]:
rdb.drivers()

{0: 'SQL Server',
 1: 'SQL Server Native Client 11.0',
 2: 'ODBC Driver 11 for SQL Server'}

In [65]:
zipi_db = rdb.conn(driver=rdb.drivers()[1],
                   server='Freshdb02.xoominfo.com',
                   database='zipi',
                   uid='sa')

Password for user sa:
········
Connected to zipi on Freshdb02.xoominfo.com.


In [67]:
query_emails = """
    select email, zoom_person_id
    from zipi_email
    where email in ({})"""
context = pd.DataFrame()
counter = 0
proportions = pd.DataFrame()
master = pd.DataFrame()

num_f = len(clickback_csv[:5])
master = pd.DataFrame()
for index, f_name in enumerate(clickback_csv[:5]):
    # mark progress
    dsp.clear_output()
    print("starting {}/{}, {}\n".format(index + 1, num_f, f_name))
    
    # read in source data
    path = os.path.join(data_dir, f_name)
    df = pd.read_csv(path, usecols=['Import_Date', 'Email_Address'], nrows=_rows)
    
    # capture line count
    f_size = df.shape[0]
    
    # downsample to desired number of records
    df = df.sample(records_per_file)
    
    # set up iteration chunk size, chunk list, and aggregator dataframe
    chunksize = 500
    chunks = [df['Email_Address'][i:i+chunksize] for i in range(0, df.shape[0], chunksize)]
    agg = pd.DataFrame()
    
    # iterate over emails while fetching zipi DB data
    for chunk in chunks:
        email_string = transform.emails_to_sql(chunk)
        temp = rdb.query(zipi_db, query_emails.format(email_string))
        agg = agg.append(temp, ignore_index=True)
    
    # only want a single record per found email
    #agg.sort_values(by='last_date', ascending=False, inplace=True)
    agg.drop_duplicates(subset='email', keep='first', inplace=True)
    
    
    joined = df.merge(agg, how='left', left_on='Email_Address', right_on='email')
    joined['file_name'] = f_name
    joined['file_size'] = f_size
    
    master = master.append(joined, ignore_index=True)

master.head(2)



starting 5/5, zoominfo_permissionpass_new_weekly_20160521_090000_Verified_20160620.csv



"\nfor i in CBsamples['Import_Date'].unique():\n    print(counter)\n    counter += 1\n    doc = CBsamples[CBsamples['Import_Date']== i]\n    email_string = narfle_the_garthok(doc['Email_Address'])\n    #email_string = narfle_the_garthok(CBsamples['Email_Address'])\n    \n    context = context.append(rdb.query(zipi_db, query_emails.format(email_string)))\n    \n    final = pd.merge(doc, context, how='left', \n                    left_on='Email_Address', right_on='email')\n    \n    #final = pd.merge(CBsamples, context, how='left', \n     #                left_on='Email_Address', right_on='email')\n    master = master.append(final)\n    \n    proportions = proportions.append([[i,\n                                      len(final[final['email'].isnull()]), \n                                      (len(final)-len(final[final['email'].isnull()]))]])\nproportions = proportions.rename(columns={0:'Import Date', 1:'Guessed', 2:'Evidence'})\nproportions\n"

In [68]:
master.head(2)

,Import_Date,Email_Address,email,zoom_person_id,file_name,file_size
0,3/12/2016,c@bubbletightusa.com,NaN,NaN,zoominfo_permissionpass_new_weekly_20160312_09...,1048575
1,3/12/2016,vasishta@jamboreeindia.com,NaN,NaN,zoominfo_permissionpass_new_weekly_20160312_09...,1048575


In [69]:
master['has_zipi'] = master['email'].notnull().map(lambda value: int(bool(value)))
pivot = master.pivot_table(index='file_name', values='has_zipi', aggfunc=np.mean)
print(pivot)

file_name
zoominfo_permissionpass_new_weekly_20160312_090000_Verified_20160331.csv    0.026
zoominfo_permissionpass_new_weekly_20160319_090000_Verified_20160419.csv    0.028
zoominfo_permissionpass_new_weekly_20160507_090000_Verified_20160530.csv    0.025
zoominfo_permissionpass_new_weekly_20160514_090000_Verified_20160615.csv    0.010
zoominfo_permissionpass_new_weekly_20160521_090000_Verified_20160620.csv    0.012
Name: has_zipi, dtype: float64


In [70]:
master.to_csv('CBemails_guessed.csv', index = False)